# PRUEBA - ING DATOS (Carga-ETL del DataWareHouse)

## Log Ejecucion

In [1]:
import datetime
from IPython.display import display, HTML
from notebook import notebookapp
servers = list(notebookapp.list_running_servers())
now = datetime.datetime.now()
display(HTML("""<style type="text/css">.generated-text {   font-family:Verdana, sans-serif; font-size:inherit;   font-style:italic;   line-height:20px;   text-align:justify;   color:#1b5727;   background-color:#ffffff;   padding:20px;   margin:auto;}</style>
<h6 class="generated-text">Proceso ejecutado por: Brayan Muñoz</h6>"""))
display ("Fecha y hora de inicio: ")
display (now.strftime("%Y-%m-%d %H:%M"))
display(servers[0])

'Fecha y hora de inicio: '

'2024-02-20 08:39'

{'base_url': '/',
 'hostname': 'localhost',
 'notebook_dir': 'C:\\Users\\braya',
 'password': False,
 'pid': 10468,
 'port': 8888,
 'secure': False,
 'sock': '',
 'token': 'cfc56ea9c00d163e9f8813d5fe0db7c04c31ccc8b93e5ec3',
 'url': 'http://localhost:8888/'}

## Librerias

In [2]:
import psycopg2 as bd

### Conexion a BD's

In [3]:
conexion = bd.connect(host="localhost", database="dvdrental", user="postgres", password="12345")

conexion_dw = bd.connect(host="localhost", database="dw_dvdrental", user="postgres", password="12345", port='5431')


## Transformacion-Carga Tabla Dw_Film

In [4]:
cur1 = conexion.cursor()
cur1.execute("""SELECT film_id, title, description, release_year , l."name" "language", rental_duration,  length, film.last_update
FROM film, "language" l where film.language_id =l.language_id""")
result = cur1.fetchall()

cursor2 = conexion_dw.cursor()

# Itera a través de los resultados y realiza la inserción
for fila in result:
    film_id, title, description, release_year, language, rental_duration, length, last_update  = fila
    cursor2.execute("""INSERT INTO dw_Film(film_id, title, description, release_year, language, rental_duration, length, last_update) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""", (film_id, title, description, release_year, language, rental_duration, length, last_update))
    conexion_dw.commit()  # Guarda los cambios en la base de datos
    
# Cierra los cursores y conexiones
cur1.close()
cursor2.close()

## Transformacion-Carga Tabla Dw_Store

In [5]:
cur1 = conexion.cursor()
cur1.execute("""SELECT  s.store_id, a.address, a.district, a.phone, c2.city, c.country, s.last_update  
                FROM country c, city c2, address a, store s
                WHERE c.country_id = c2.country_id and c2.city_id =a.city_id and a.address_id = s.address_id """)
result = cur1.fetchall()

cursor2 = conexion_dw.cursor()

# Itera a través de los resultados y realiza la inserción
for fila in result:
    store_id, address, district, phone, city, country, last_update  = fila
    cursor2.execute("""INSERT INTO dw_Store(store_id, address, district, phone, city, country, last_update) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)""", (store_id, address, district, phone, city, country, last_update))
    conexion_dw.commit()  # Guarda los cambios en la base de datos
    
# Cierra los cursores y conexiones
cur1.close()
cursor2.close()

## Transformacion-Carga Tabla Dw_Customer

In [6]:
cur1 = conexion.cursor()
cur1.execute("""SELECT  b.customer_id, b.first_name, b.last_name, b.email, b.active, a.address, a.district, a.phone, c2.city, c.country, b.last_update  
                FROM country c, city c2, address a, customer b
                WHERE c.country_id = c2.country_id and c2.city_id =a.city_id and a.address_id = b.address_id """)
result = cur1.fetchall()

cursor2 = conexion_dw.cursor()

# Itera a través de los resultados y realiza la inserción
for fila in result:
    customer_id,first_name, last_name, email, active, address, district, phone, city, country, last_update  = fila
    cursor2.execute("""INSERT INTO dw_Customer(customer_id,first_name, last_name, email, active, address, district, phone, city, country, last_update) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", (customer_id,first_name, last_name, email, active, address, district, phone, city, country, last_update))
    conexion_dw.commit()  # Guarda los cambios en la base de datos
    
# Cierra los cursores y conexiones
cur1.close()
cursor2.close()

## Transformacion-Carga Tabla Dw_Staff

In [7]:
cur1 = conexion.cursor()
cur1.execute("""SELECT  s.staff_id , s.first_name, s.last_name, s.email, s.active, s.username, s."password", a.address, a.district, a.phone, c2.city, c.country, s.last_update  
                FROM country c, city c2, address a, staff s 
                WHERE c.country_id = c2.country_id and c2.city_id =a.city_id and a.address_id = s.staff_id """)
result = cur1.fetchall()

cursor2 = conexion_dw.cursor()

# Itera a través de los resultados y realiza la inserción
for fila in result:
    staff_id,first_name, last_name, email, active, username, password, address,district, phone, city, country, last_update  = fila
    cursor2.execute("""INSERT INTO dw_Staff(staff_id,first_name, last_name, email, active, username, password, address,district, phone, city, country, last_update) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", (staff_id,first_name, last_name, email, active, username, password, address,district, phone, city, country, last_update))
    conexion_dw.commit()  # Guarda los cambios en la base de datos
    
# Cierra los cursores y conexiones
cur1.close()
cursor2.close()

## Transformacion-Carga Tabla Dw_Date

In [8]:
cur1 = conexion.cursor()
cur1.execute("""SELECT DISTINCT to_char(P.payment_date ,'YYYY/MM/DD') Fecha, to_char(P.payment_date ,'YYYY') yea,
                to_char(P.payment_date,'MONTH') mes,to_char(P.payment_date ,'DD') dia 
                FROM payment p""")
result = cur1.fetchall()

cursor2 = conexion_dw.cursor()

# Itera a través de los resultados y realiza la inserción
for fila in result:
    Fecha, yea, mes, dia  = fila
    cursor2.execute("""INSERT INTO dw_Date(Date, year, month, day) 
    VALUES (%s, %s, %s, %s)""", (Fecha, yea, mes, dia))
    conexion_dw.commit()  # Guarda los cambios en la base de datos
    
# Cierra los cursores y conexiones
cur1.close()
cursor2.close()

## Transformacion-Carga Tabla Dw_Sales

In [9]:
cur1 = conexion.cursor()
cur1.execute("""SELECT DISTINCT p.payment_id, s.store_id,  s.staff_id, c.customer_id,  f.film_id, to_char(P.payment_date ,'YYYY/MM/DD') Fecha, p.amount  
                FROM film f, inventory i, rental r , payment p, customer c, staff s, store s2 
                WHERE f.film_id =i.film_id and i.inventory_id = r.inventory_id and r.rental_id = p.rental_id and p.customer_id =c.customer_id and p.staff_id =s.staff_id and s.store_id =s2.store_id  """)
result = cur1.fetchall()

cursor2 = conexion_dw.cursor()

# Itera a través de los resultados y realiza la inserción
for fila in result:
    payment_id, store_id, staff_id, customer_id, film_id, Fecha, amount  = fila
    cursor2.execute("""INSERT INTO dw_Sales(sales_id, store_fk, staff_fk, customer_fk, film_fk, date_fk, amount) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)""", (payment_id, store_id, staff_id, customer_id, film_id, Fecha, amount))
    conexion_dw.commit()  # Guarda los cambios en la base de datos
    
# Cierra los cursores y conexiones
cur1.close()
cursor2.close()

In [10]:
conexion.close()

conexion_dw.close()

# FIN :)